Things to Import

In [ ]:
%matplotlib inline
import random
import requests
import matplotlib.pyplot as plt

Function to get Data from IEX API

In [ ]:
def getData(ticker, attribute):
    URL = "https://api.iextrading.com/1.0/stock/" + ticker.lower() + "/chart/5y"
    PARAMS = {} #  {'param':"value"}
    r = requests.get(url = URL, params = PARAMS)
    data = r.json()
    return [float(x[attribute]) for x in data]

In [ ]:
AMD_close_data = getData('AMD', 'close')

print(AMD_close_data[0:100])

Function to convert from a time series to percentage change series

In [ ]:
def percentChanges(numbers):
    if len(numbers) > 1:
        return [100.0 * ((numbers[x+1] - numbers[x])/numbers[x]) for x in range(len(numbers) - 1)]
    else:
        return []

In [ ]:
AMD_percent_changes = percentChanges(AMD_close_data)

print(AMD_percent_changes[0:100])

Plot function to make life easier

In [ ]:
def plot(chart_type, numbers, ylabel, xlabel):
    if chart_type == 'line':
        plt.plot(numbers)
    elif chart_type == 'histogram':
        plt.hist(numbers, 100)
    plt.ylabel(ylabel)
    plt.xlabel(xlabel)
    plt.show()

In [ ]:
plot('line', AMD_close_data, 'Price', 'Day')

In [ ]:
plot('histogram', AMD_percent_changes, 'Frequency', '% Change')

In [ ]:
def generate_series(start, percent_change_possibilities, length):
    generated = [start]
    for i in range(length):
        generated.append(generated[-1] * (1 + random.choice(percent_change_possibilities)/100.0))
    return generated

In [ ]:
starting_price = 100
days_to_fake = 1000

fake_stock = generate_series(starting_price, AMD_percent_changes, days_to_fake)

plot('line', fake_stock, 'Price', 'Day')

Create a simple strategy

In [ ]:
def strategy_1(numbers, tf, mult):
    series_range = []
    high_bounds = []
    low_bounds = []
    
    position = None
    stop_loss = None
    current_trade = 0
    account = 0
    account_history = []
    
    for i in range(len(numbers)):
        if i < tf:
            series_range.append(None)
            high_bounds.append(None)
            low_bounds.append(None)
            account_history.append(account)
        else:
            subset = numbers[i-tf:i]
            series_range.append(max(subset) - min(subset))
            high_bounds.append(numbers[i] + mult * series_range[i])
            low_bounds.append(numbers[i] - mult * series_range[i])
            
            if position == None:
                position = random.choice(['long', 'short'])
                if position == 'long':
                    stop_loss = low_bounds[i]
                    current_trade = numbers[i]
#                     print("ENTER LONG @ " + str(numbers[i]))
                elif position == 'short':
                    stop_loss = high_bounds[i]
                    current_trade = numbers[i]
#                     print("ENTER SHORT @ " + str(numbers[i]))
                account_history.append(account)
            elif position == 'long':
                stop_loss = max(low_bounds[i], stop_loss)
                if numbers[i] < stop_loss:
                    position = None
                    account += numbers[i] - current_trade
#                     print("EXIT LONG @ " + str(numbers[i]) + " P/L: " + str(numbers[i] - current_trade))
                account_history.append(account + numbers[i] - current_trade)
            elif position == 'short':
                stop_loss = min(high_bounds[i], stop_loss)
                if numbers[i] > stop_loss:
                    position = None
                    account -= numbers[i] - current_trade
#                     print("EXIT SHORT @ " + str(numbers[i]) + " P/L: " + str(current_trade - numbers[i]))
                account_history.append(account - numbers[i] + current_trade)
#             print(position, numbers[i], stop_loss)

    return [high_bounds, numbers, low_bounds, account_history]

Test Strategy on our fake stock

In [ ]:
sfs = strategy_1(fake_stock, 50, 1.5)

for element in sfs[:3]:
    plt.plot(element)
plt.title("Strategy 1")
plt.ylabel("Price")
plt.xlabel("Day")
plt.show()

plt.plot(sfs[3])
plt.ylabel("P/L")
plt.xlabel("Day")
plt.show()

Getting some sample data

In [ ]:
tickers = ['aapl', 'ge', 'ko', 'snap', 'fb', 'jnj']
percent_changes = []
for ticker in tickers:
    percent_changes += percentChanges(getData(ticker, 'close'))

Testing the strategy on many random stocks

In [ ]:
account_lines = []
for i in range(1000):
    fake_stock = generate_series(100, percent_changes, 1000)
    sfs = strategy_1(fake_stock, 50, 1.5)
    account_lines.append(sfs[3])

avg_acct = []
for i in range(len(account_lines[0])):
    total = 0
    for acct in account_lines:
        total += acct[i]
    avg_acct.append(total / len(account_lines))

plt.plot(avg_acct)
plt.ylabel("P/L")
plt.xlabel("Day")
plt.show()